In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings

# from langchain.vectorstores.chroma import Chroma
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/1984_chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    # chain_type="stuff",
    # chain_type="refine",
    # chain_type="map_reduce",
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

# chain.run("Where dose Winston live?")
print(chain.run("Describe Victory Mansions"))
# print(chain.run("Who are main characters?"))

c:\Users\woosu\Dev\python\fullstack-gpt\env\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Victory Mansions is described as a place with no windows and impossible to enter except on official business. The streets leading up to its outer barriers are guarded by gorilla-faced guards in black uniforms. Winston sacrificed his lunch to leave the Ministry and went into a tiny kitchen with minimal food. The kitchen contains a bottle of colorless liquid labeled VICTORY GIN, which gives off a sickly smell and burns like nitric acid when consumed. The living-room has a small table to the left of the telescreen, with a drawer containing a penholder, ink bottle, and a blank book with a red back and marbled cover.


In [ ]:
# results = vectorstore.similarity_search("where dose winston live")
results = vectorstore.similarity_search("Where dose Winston live???")
results